In [56]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy import sparse 
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import chi2, SelectPercentile,SelectKBest
import gc
from sklearn.metrics import log_loss

In [ ]:
train = pd.read_pickle(FILE.train_final.value)
test = pd.read_pickle(FILE.test_final.value)
data = pd.concat([train, test],sort=False)

In [47]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [48]:
data['user_tags_encoded'] = le.fit_transform(data['user_tags'].astype(str))

In [49]:
def compute_the_id(row, joinby='-'):
    the_str = ''
    for val in row.values:
        the_str += joinby+str(val)
#         if val == 95764:  95764 => user tags is NAN
#             return np.nan 
    return the_str[1:]

In [50]:
user_id_cols = ['city','osv','make','model','user_tags_encoded']
data['user_id'] = data[user_id_cols].apply(lambda row: compute_the_id(row) , axis=1)

# Codes

In [51]:
from scipy import sparse
from scipy.sparse.linalg import svds

def svd_two_cols(df, col1_name, col2_name, n_component):
    data = np.ones(len(df))
    le = LabelEncoder()
    df = df.copy()
    df[col1_name] = le.fit_transform(df[col1_name].astype(str))
    df[col2_name] = le.fit_transform(df[col2_name].astype(str))
    col1 = df[col1_name].values
    col2 = df[col2_name].values

    col1_cnt = max(col1)+1
    col2_cnt = max(col2)+1
    print(col1_cnt, col2_cnt)

    rating = sparse.coo_matrix((data, (col1, col2)))
    rating = (rating > 0) * 1.0

    [u, s, vt] = svds(rating, k=n_component)
    print(u.shape)
    print(s.shape)
    print(vt.shape)
    print(s[::-1])
    col1_col2 = np.diag(s[::-1])

    col1_topics = pd.DataFrame(u[:, ::-1])
    col1_topics.columns = ['{}_{}_svd_{}'.format(col1_name, col2_name, i) for i in range(n_component)]
    col1_embed = col1_topics.values
    col1_topics[col1_name] = range(col1_cnt)

    df = df.merge(col1_topics, on=col1_name, how='left')

    col2_topics = pd.DataFrame(vt.transpose()[:, ::-1])
    col2_topics.columns = ['{}_{}_svd_{}'.format(col2_name, col1_name, i) for i in range(n_component)]
    col2_embed = col2_topics.values
    col2_topics[col2_name] = range(col2_cnt)

    df = df.merge(col2_topics, on=col2_name, how='left')

    return df, col1_col2, col1_embed, col2_embed

In [52]:
df, user_ad_for_dot, user_ad_user_embed, user_ad_ad_embed = svd_two_cols(data, 'user_id', 'adid', 16)
df, model_ad_for_dot, model_ad_model_embed, model_ad_ad_embed = svd_two_cols(df, 'model', 'adid', 16)
df, tag_ad_for_dot, tag_ad_tag_embed, tag_ad_ad_embed = svd_two_cols(df, 'user_tags_encoded', 'adid', 16)

2327425 2221
(2327425, 16)
(16,)
(16, 2221)
[350.12076538 345.73541756 340.3279445  301.66167836 297.86039802
 289.18351978 272.05388218 261.50472411 252.33731351 225.78723232
 220.6932322  216.59175075 215.55702342 198.29491748 192.99452896
 187.70081697]
20332 2221
(20332, 16)
(16,)
(16, 2221)
[333.47204852 147.80008126 105.99190644  94.08657677  87.33754918
  81.82397571  74.31152966  66.63746345  54.68527316  53.30165307
  48.79600707  48.35081725  43.01864511  41.91258192  39.3089441
  38.75585021]
1345254 2221
(1345254, 16)
(16,)
(16, 2221)
[286.05661523 249.19831531 238.67673108 234.67312317 224.08247975
 213.68687174 203.34021612 201.84383008 193.92289273 179.12998691
 172.16582183 167.21125061 158.79903506 151.83018445 150.17206646
 142.12058069]


In [54]:
df_save = df[['instance_id']].copy()
for col in df:
    if 'svd' in col:
        df_save[col] = df[col].values

In [55]:
df_save.to_pickle('../../data/features/agg/svd_raw.pkl')

In [32]:
df = data.copy()
col1_name = 'user_id'
col2_name = 'adid'
data = np.ones(len(df))
le = LabelEncoder()
df = df.copy()
df[col1_name] = le.fit_transform(df[col1_name].astype(str))
df[col2_name] = le.fit_transform(df[col2_name].astype(str))
col1 = df[col1_name].values
col2 = df[col2_name].values


In [36]:
col1_cnt = max(col1)+1
col2_cnt = max(col2)+1
print(col1_cnt, col2_cnt)
rating = sparse.lil_matrix((col1_cnt, col2_cnt))
for v1,v2 in tqdm(zip(col1,col2)):
    rating[v1,v2] += 1

2327425 2221


3072915it [00:18, 166142.01it/s]


In [38]:
rating = sparse.coo_matrix(rating)
n_component = 32
[u, s, vt] = svds(rating, k=n_component)
print(u.shape)
print(s.shape)
print(vt.shape)

(2327425, 32)
(32,)
(32, 2221)


In [8]:
df, user_ad_for_dot, user_ad_user_embed, user_ad_ad_embed = svd_two_cols(df, 'user_id', 'adid', 32)
df, model_ad_for_dot, model_ad_model_embed, model_ad_ad_embed = svd_two_cols(df, 'model', 'adid', 32)
df, tag_ad_for_dot, tag_ad_tag_embed, tag_ad_ad_embed = svd_two_cols(df, 'user_tags_sorted', 'adid', 32)
print(df.shape)

NameError: name 'df' is not defined